# Dining philosophers problem

Five silent philosophers sit at a round table with bowls of spaghetti. Forks are placed between each pair of adjacent philosophers.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7b/An_illustration_of_the_dining_philosophers_problem.png/463px-An_illustration_of_the_dining_philosophers_problem.png)

Each philosopher must alternately think and eat. However, a philosopher can only eat spaghetti when they have both left and right forks. Each fork can be held by only one philosopher and so a philosopher can use the fork only if it is not being used by another philosopher. After an individual philosopher finishes eating, they need to put down both forks so that the forks become available to others. A philosopher can take the fork on their right or the one on their left as they become available, but cannot start eating before getting both forks.

Eating is not limited by the remaining amounts of spaghetti or stomach space; an infinite supply and an infinite demand are assumed.

The problem is how to design a discipline of behavior (a concurrent algorithm) such that no philosopher will starve; i.e., each can forever continue to alternate between eating and thinking, assuming that no philosopher can know when others may want to eat or think.

## Compute the transition matrix

Each philospher acts according to the following graph of states:
<img src="./dining_philosophers.png" alt="Drawing" style="width: 70%;"/>

We now want to know, given a set of states like $s = (s_{phil1}, s_{phil2}, s_{phil3}) = (0, 1, 2)$ $\dots$

In [20]:
from pprint import pprint
import itertools as itt
import numpy as np

# Fix constants at the top, so we always know where to go to edit things.
h, e = 0.05, 0.75

num_phil = 3
num_states = 10

In [21]:
def compute_successor(qi, qi_prev, qi_foll, h, e):
    """This function returns the state and probability of a philosopher after an iteration,
    according to the status of its neighbors.
    """
    
    def has_lf(q):
        return q in (4, 6, 8, 9)
    def has_rf(q):
        return q in (5, 7, 8, 9)
    def needs_lf(q):
        return q in (2, 5)
    def needs_rf(q):
        return q in (3, 4)
    
    if qi == 0:
        return [([0], 1-h), ([1], h)]
    if qi == 1:
        return [([2], 0.5), ([3], 0.5)]
    if qi == 2:
        if has_rf(qi_prev):
            return [([2], 1)]
        if not has_rf(qi_prev) and not needs_rf(qi_prev):
            return [([4], 1)]
        if not has_rf(qi_prev) and needs_rf(qi_prev):
            return [([2], 1)]
    if qi == 3:
        if has_lf(qi_foll):
            return [([3], 1)]
        return [([5], 1)]
    if qi == 4:
        if has_lf(qi_foll):
            return [([6], 1)]
        return [([8], 1)]
    if qi == 5:
        if has_rf(qi_prev):
            return [([7], 1)]
        return [([8], 1)]
    if qi in (6,7):
        return [([1], 1)]
    
    if qi == 8:
        return [([9], 1)]

    if qi == 9:
        return [([9], e), ([0], 1-e)]

In [4]:
compute_successor(1, 2, 3, 0.05, 0.75)

[([2], 0.5), ([3], 0.5)]

In [5]:
compute_successor(8, 8, 8, 0.05, 0.75)

[([9], 1)]

In [6]:
def cartesian_prod(list_to_be_extended, list_to_add):
    """This function computes the cartesian product between two lists made like:
    
    list_to_be_extended = [([a, b], 0.13), ([a, c], 0.24), ([b, c], 0.45)]
    list_to_add = [([x], 0.12), ([y], 0.35)]
    
    where {a, b, c, x, y} are possible states, coming from the graph previously
    presented, and the associated number in the pair is the probability of the
    philosopher to follow/act according to those states.
    
    Please note that `list_to_be_extended` could be an empty list at the beginning
    of the iteration process and, in that case, the `list_to_add` list has to be
    returned.
    """
    
    if not list_to_be_extended:
        return list_to_add
    
    list_with_products_to_return = []
    
    for single_state_to_add, prob_single_state in list_to_add:
        for list_of_states, prob_sequence_states in list_to_be_extended:
            list_with_products_to_return.append((list_of_states + single_state_to_add,
                                                prob_single_state * prob_sequence_states))

    return list_with_products_to_return

    
print("Perform some simple tests:\n")

test_empty_list = []
test_to_add_list = [([3], 0.1), ([4], 0.2)]
print("empty list")
pprint(cartesian_prod(test_empty_list, test_to_add_list))

test_midlen_list = [([0], 0.2), ([1], 0.3)]
print("\n2 elements single-state list")
pprint(cartesian_prod(test_midlen_list, test_to_add_list))

test_long_list = [([0, 0], 0.2), ([0, 1], 0.3)]
print("\n2 elements double-state list")
pprint(cartesian_prod(test_long_list, test_to_add_list))

Perform some simple tests:

empty list
[([3], 0.1), ([4], 0.2)]

2 elements single-state list
[([0, 3], 0.020000000000000004),
 ([1, 3], 0.03),
 ([0, 4], 0.04000000000000001),
 ([1, 4], 0.06)]

2 elements double-state list
[([0, 0, 3], 0.020000000000000004),
 ([0, 1, 3], 0.03),
 ([0, 0, 4], 0.04000000000000001),
 ([0, 1, 4], 0.06)]


In [12]:
def compute_sequence_states(input_state, num_phil):
    """Given a sequence of states and the number of philosophers, return the successors.
    
    The intermediate states (e.g. input [0, 0, 0], intermediate [0, 0] or [1, 0]) are
    not returned.
    """
    _intermediate_states = []
    states_to_return = []
    
    for ind, _ in enumerate(input_state):
        single_successor_state = compute_successor(input_state[ind], input_state[ind-1], input_state[(ind+1)%len(input_state)], h, e)
        successor_states = cartesian_prod(_intermediate_states, single_successor_state)

        for states, prob in successor_states:
            if len(states) == num_phil:
                states_to_return.append((states, prob))
            else:
                _intermediate_states.append((states, prob))
            
    return states_to_return

In [13]:
input_state = [0] * num_phil

state_list = compute_sequence_states(input_state, num_phil)
pprint(state_list)

[([0, 0], 0.9025),
 ([1, 0], 0.0475),
 ([0, 1], 0.0475),
 ([1, 1], 0.0025000000000000005)]


## Naive implementation

Compute the transition matrix for all the states, even for those not reachable (e.g. $philosopher_i$ eating && $philosopher_{i+1}$ eating, ..) and then remove those states not reachable.

A similar implementation could compute the reachable states and then calculate the probabilities for each reachable state.

## Smart implementation

We chose to start from a known reachable state, which happens when all philosophers are thinking, so when all of them are on state $0$: $[0_{ph.1}, 0_{ph.2}, \dots, 0_{ph.N}]$.

From this known "safe" state, we calculate the reachable states in a single step, which, for a 2-philosophers problem, are $\{[0, 0], \; [0, 1], \; [1, 0], \; [1, 1]\}$, along with their probabilities.

Then we insert those states directly into the transition matrix along with their probabilities, and call recursively the same function on the just discovered states.

With this approach, there's no need to calculate impossible or un-reachable states.

In [9]:
"""
We need functions to convert from a state (e.g. '[1, 2, 3]') to an index (e.g. '456'),
because cells of the transition matrix can be accessed only with indices.

The following two functions should be invertible, which means converting from a state
to an index to a state should produce the same original state.
"""

def state_to_index(state):
    return sum((single_state * num_states**(num_phil-1-index) for index, single_state in enumerate(state)))

"""
The previous generator-comprehension (similar to a list-comprehension, but faster and
more powerful, since it's lazily evaluated https://en.wikipedia.org/wiki/Python_syntax_and_semantics#Generators )
is practically identical to the following lines:
"""
#     tot = 0
#     for index, singol_state in enumerate(state):
#         tot += singol_state * num_states**(num_phil-1-index) 
#     return tot


def index_to_state(index):
    """Converts an index into a sequence of states, which may not be 10-based integers.
    
    Extracts elements starting from the highest power; then move to extract from the
    remainder of the division until all the digits have been processed.
    """
    
    # Here we cannot use a *-comprehension, since we also need to modify the `index' value.
    # Or maybe we could do it, but sometimes "Beautiful is better than ugly" and
    # "Flat is better than nested" (from: https://en.wikipedia.org/wiki/Zen_of_Python )
    sequence = []
    for power in range(num_phil):
        sequence.append(index // num_states**(num_phil-1-power))
        index %= num_states**(num_phil-1-power)
    return sequence

print(state_to_index(index_to_state(123)) == 123)
print(state_to_index(index_to_state(state_to_index(index_to_state(123)) + 1)) == 123+1)

True
True


In [14]:
def explore_and_build_transition_matrix(states_matrix, start_state):
    """Given the transition matrix and a state as input, calculate the successor
    of the states, place them into the matrix and call recursively this function
    only when the pair (actual_state, successor_state) has not been previously
    entered into the matrix.
    """
    index_start_state = state_to_index(start_state)

    sequence_following_states = compute_sequence_states(start_state, num_phil)
    
    for state, prob_associated in sequence_following_states:

        # This check works since our initial matrix is made of zeros; if we would
        # use a sparse matrix, as in the following implementation, this check could
        # be different. We use it to make sure that we don't loop when searching for
        # successors of states.
        if states_matrix[index_start_state, state_to_index(state)] == 0:
            states_matrix[index_start_state, state_to_index(state)] = prob_associated
            
            explore_and_build_transition_matrix(states_matrix, state)

In [15]:
initial_safe_combination = [0] * num_phil

In [16]:
transition_matrix = np.zeros([num_states**num_phil, num_states**num_phil])

explore_and_build_transition_matrix(transition_matrix, initial_safe_combination)

In [18]:
# The sum of probabilities on the rows must be either 1 (given any state, we must go
# to some other states in a single iteration) or 0, if the state is not admissible
# in our system (such as "all philosophers are eating at the same time").
np.sum(transition_matrix, axis=1)

array([1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0.])

## Smarter implementation (with sparse matrix)

Notice how many bytes are needed to store our transition matrix:

In [114]:
transition_matrix.nbytes

80000

In [19]:
print("On a", transition_matrix.shape, "matrix, there are only", np.count_nonzero(transition_matrix),
      "non-zero elements, which is", np.count_nonzero(transition_matrix)/np.prod(transition_matrix.shape),
      "% of the total.")

On a (100, 100) matrix, there are only 106 non-zero elements, which is 0.0106 % of the total.


We could greatly reduce the size of the transition matrix and increase the performance of our system by using a transition matrix made of a sparse matrix, which means storing only the elements different than 0.